In [1]:
import pandas
import datetime
import numpy
import math
from shapely.geometry import Point

In [2]:
gnaf_path = "/Users/danielcorcoran/Desktop/GNAF VIC MAY18/OUTPUT/GNAF_MERGED_SUBSET.csv"
nodes_dpc = "/Users/danielcorcoran/Desktop/Tableau (mac)/Tableau Public Projects/Vic Crash Stats (Extracted 4-5-2018)/Vicroads Data/NODES_DPC.csv"

In [3]:
gnaf_data = pandas.read_csv(gnaf_path)

In [4]:
node_data = pandas.read_csv(nodes_dpc)

In [5]:
node_data.head()

,NODE_ROW_INDEX,ACCIDENT_NO,NODE_ID,NODE_TYPE,AMG_X,AMG_Y,LGA_NAME,Lga Name All,Region Name,Deg Urban Name,Lat,Long,Postcode No,node_row_index,polygon_index,lga_code,lga_name
0,0,T20060000010,43078,I,2519154.655,2390265.154,DANDENONG,DANDENONG,METROPOLITAN SOUTH EAST REGION,MELB_URBAN,-37.98862,145.21805,3175,0,25,22670,Greater Dandenong (C)
1,1,T20060000018,29720,N,2524272.743,2389996.817,CASEY,CASEY,METROPOLITAN SOUTH EAST REGION,MELB_URBAN,-37.99092,145.27632,3804,1,13,21610,Casey (C)
2,2,T20060000022,203074,N,2487321.693,2345019.925,MORNINGTON PENINSULA,MORNINGTON PENINSULA,METROPOLITAN SOUTH EAST REGION,RURAL_VICTORIA,-38.39632,144.85489,3939,2,52,25340,Mornington Peninsula (S)
3,3,T20060000023,55462,I,2512734.120,2390214.959,DANDENONG,"KINGSTON,DANDENONG",METROPOLITAN SOUTH EAST REGION,MELB_URBAN,-37.98918,145.14496,3173,3,34,23430,Kingston (C) (Vic.)
4,4,T20060000026,202988,N,2488777.662,2347612.069,MORNINGTON PENINSULA,MORNINGTON PENINSULA,METROPOLITAN SOUTH EAST REGION,MELB_URBAN,-38.37299,144.87159,3940,4,52,25340,Mornington Peninsula (S)


In [6]:
gnaf_data.iloc[748115]

ROW_INDEX                              748115
ADDRESS_DETAIL_PID             GAVIC421916814
POSTCODE                                 3175
ADDRESS_SITE_PID                    422053715
STREET_LOCALITY_PID                VIC1958844
GNAF_PROPERTY_PID                      578458
ADDRESS_DEFAULT_GEOCODE_PID           1278932
LONGITUDE                             145.218
LATITUDE                             -37.9886
STREET_NAME                            FOSTER
STREET_TYPE_CODE                       STREET
Name: 748115, dtype: object

In [7]:
gnaf_data.describe()

,ROW_INDEX,POSTCODE,ADDRESS_SITE_PID,GNAF_PROPERTY_PID,ADDRESS_DEFAULT_GEOCODE_PID,LONGITUDE,LATITUDE
count,3.714347e+06,3.714347e+06,3.714347e+06,3.225817e+06,3.714347e+06,3.714347e+06,3.714347e+06
mean,1.857173e+06,3.310223e+03,4.404218e+08,1.093031e+08,1.998742e+08,1.449537e+02,-3.772886e+01
std,1.072240e+06,3.073091e+02,7.263568e+07,1.351617e+08,7.428647e+08,8.188082e-01,5.868097e-01
min,0.000000e+00,3.000000e+03,4.108684e+08,1.600000e+02,1.237895e+06,1.409657e+02,-3.903061e+01
25%,9.285865e+05,3.073000e+03,4.203861e+08,1.487469e+06,2.204810e+06,1.448096e+02,-3.797183e+01
50%,1.857173e+06,3.178000e+03,4.215815e+08,5.032395e+07,3.171878e+06,1.450054e+02,-3.782319e+01
75%,2.785760e+06,3.522000e+03,4.243285e+08,2.073622e+08,4.138500e+06,1.451754e+02,-3.771681e+01
max,3.714346e+06,3.996000e+03,7.151561e+08,4.279787e+08,3.010403e+09,1.498257e+02,-3.406838e+01


## Create reference lists containing unique ids

In [8]:
gnaf_index = list(gnaf_data["ROW_INDEX"])

In [9]:
node_index = list(node_data["NODE_ROW_INDEX"])

## Process

start = datetime.datetime.now()

dresults = {}
iresults = {}

for nindex in range(3):
    
    print("Starting to process nindex :", nindex)
    
    #store coords for nindex

    node_long = node_data.loc[nindex,"Long"]
    node_lat = node_data.loc[nindex,"Lat"]
    
    dresults[nindex] = 99999999999
    
    for gindex in range(gnaf_data.shape[0]):
        
        #set coordinates
        
        gnaf_long = gnaf_data.loc[gindex,"LONGITUDE"]
        gnaf_lat = gnaf_data.loc[gindex,"LATITUDE"]
        
        #calculate diagonal distance
        
        long_diff = node_long - gnaf_long
        lat_diff =  node_lat - gnaf_lat
        
        calculated_distance = math.sqrt(long_diff**2 + lat_diff**2)
        
        #print(long_diff,lat_diff,calculated_distance,sep="__")
        
        if calculated_distance < dresults[nindex]:
            print("Updated nindex",nindex,"with gindex",gindex,".")
            dresults[nindex] = calculated_distance
            iresults[nindex] = gindex
            



In [10]:
node_points = []
gnaf_points = []

for index in range(node_data.shape[0]):
    
    node_long = node_data.loc[index,"Long"]
    node_lat = node_data.loc[index,"Lat"]
    node_points.append((node_long,node_lat))
    
print("NODE POINTS LIST LENGTH", len(node_points))

for index in range(gnaf_data.shape[0]):
    
    gnaf_long = gnaf_data.loc[index,"LONGITUDE"]
    gnaf_lat = gnaf_data.loc[index,"LATITUDE"]
    gnaf_points.append((gnaf_long,gnaf_lat))
    
print("GNAF POINTS LIST LENGTH",len(gnaf_points))

NODE POINTS LIST LENGTH 163913
GNAF POINTS LIST LENGTH 3714347


In [11]:
start = datetime.datetime.now()

dresults = {}
iresults = {}


for nindex in range(20):
    
    print("Starting to process nindex :", nindex)
    
    dresults[nindex] = 99999999999
    
    for gindex in range(len(gnaf_points)):
        
        calculated_distance = math.sqrt((node_points[nindex][0] - gnaf_points[gindex][0])**2 + (node_points[nindex][1] - gnaf_points[gindex][1])**2)
        
        if calculated_distance < dresults[nindex]:
            print("Updated nindex",nindex,"with gindex",gindex,".")
            dresults[nindex] = calculated_distance
            iresults[nindex] = gindex
            
print(datetime.datetime.now() - start)

Starting to process nindex : 0
Updated nindex 0 with gindex 0 .
Updated nindex 0 with gindex 4 .
Updated nindex 0 with gindex 66 .
Updated nindex 0 with gindex 67 .
Updated nindex 0 with gindex 113 .
Updated nindex 0 with gindex 217 .
Updated nindex 0 with gindex 377 .
Updated nindex 0 with gindex 449 .
Updated nindex 0 with gindex 675 .
Updated nindex 0 with gindex 2112 .
Updated nindex 0 with gindex 12468 .
Updated nindex 0 with gindex 12610 .
Updated nindex 0 with gindex 60775 .
Updated nindex 0 with gindex 103889 .
Updated nindex 0 with gindex 303284 .
Updated nindex 0 with gindex 748115 .
Starting to process nindex : 1
Updated nindex 1 with gindex 0 .
Updated nindex 1 with gindex 4 .
Updated nindex 1 with gindex 54 .
Updated nindex 1 with gindex 113 .
Updated nindex 1 with gindex 166 .
Updated nindex 1 with gindex 217 .
Updated nindex 1 with gindex 4701 .
Updated nindex 1 with gindex 7018 .
Updated nindex 1 with gindex 7093 .
Updated nindex 1 with gindex 9402 .
Updated nindex 1 wi

Updated nindex 12 with gindex 228206 .
Updated nindex 12 with gindex 233487 .
Updated nindex 12 with gindex 412762 .
Updated nindex 12 with gindex 1440078 .
Starting to process nindex : 13
Updated nindex 13 with gindex 0 .
Updated nindex 13 with gindex 54 .
Updated nindex 13 with gindex 75 .
Updated nindex 13 with gindex 96 .
Updated nindex 13 with gindex 103 .
Updated nindex 13 with gindex 175 .
Updated nindex 13 with gindex 2171 .
Updated nindex 13 with gindex 2172 .
Updated nindex 13 with gindex 628612 .
Updated nindex 13 with gindex 938179 .
Updated nindex 13 with gindex 2108246 .
Updated nindex 13 with gindex 2383824 .
Starting to process nindex : 14
Updated nindex 14 with gindex 0 .
Updated nindex 14 with gindex 4 .
Updated nindex 14 with gindex 66 .
Updated nindex 14 with gindex 67 .
Updated nindex 14 with gindex 113 .
Updated nindex 14 with gindex 150 .
Updated nindex 14 with gindex 217 .
Updated nindex 14 with gindex 377 .
Updated nindex 14 with gindex 449 .
Updated nindex 14 

In [12]:
dresults

{0: 0.00033405388787777364,
 1: 0.0010043843168738437,
 2: 0.000253980319317985,
 3: 0.0007741779387959148,
 4: 0.0001763413309555592,
 5: 0.0002667852583984449,
 6: 0.0004530604053607204,
 7: 0.0003596845840522413,
 8: 0.0006312162862255983,
 9: 0.00016835973390553395,
 10: 0.00033924917096094415,
 11: 0.00043603784239101714,
 12: 0.0006865801479768095,
 13: 0.0004789411789671457,
 14: 0.0004549767154329235,
 15: 0.00015885353789713897,
 16: 0.0008432429617387178,
 17: 0.0009462314727217048,
 18: 0.0003677839039432309,
 19: 0.0006882503882315602}

In [13]:
iresults

{0: 748115,
 1: 3316705,
 2: 69424,
 3: 1672264,
 4: 3507205,
 5: 1208074,
 6: 3473685,
 7: 1648621,
 8: 179735,
 9: 1576722,
 10: 482029,
 11: 2122716,
 12: 1440078,
 13: 2383824,
 14: 294289,
 15: 2349470,
 16: 125669,
 17: 940554,
 18: 2724762,
 19: 3331052}

## Point objects in lists method

node_points = []
gnaf_points = []

for index in range(node_data.shape[0]):
    
    node_long = node_data.loc[index,"Long"]
    node_lat = node_data.loc[index,"Lat"]
    point = Point(node_long, node_lat)
    node_points.append(point)
    
print(len(node_points))
    
for index in range(gnaf_data.shape[0]):
    
    gnaf_long = gnaf_data.loc[index,"LONGITUDE"]
    gnaf_lat = gnaf_data.loc[index,"LATITUDE"]
    point = Point(gnaf_long, gnaf_lat)
    gnaf_points.append(point)

print(len(gnaf_points))
    


start = datetime.datetime.now()


dresults = {}
iresults = {}

for nindex in range(3):
    
    print("Starting to process nindex :", nindex)
    
    base_point = node_points[nindex]
    
    dresults[nindex] = 99999999999
    
    for gindex in range(len(gnaf_points)):
        
        #set coordinates
        
        gnaf_point = gnaf_points[gindex]
        
        #calculate diagonal distance
        
        calculated_distance = base_point.distance(gnaf_point)
        
        if calculated_distance < dresults[nindex]:
            print("Updated nindex",nindex,"with gindex",gindex,".")
            dresults[nindex] = calculated_distance
            iresults[nindex] = gindex
            
print(datetime.datetime.now()-start)